In [33]:
import os

import pandas as pd

import yfinance as yf
from datetime import datetime, timedelta

In [34]:
# Load ticker symbols from nasdaq screener csv file
def load_tickers(filepath):
    df = pd.read_csv(filepath)
    return df.iloc[:, 0].tolist()

In [37]:
# Fetch historical data for a given ticker
def fetch_stock_data(ticker):
    # Calculate the date 5 years ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*5)
    
    # Fetch historical data
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return data

In [36]:
# Load ticker symbols
tickers = load_tickers('nasdaq_screener.csv')

# Create a subfolder for the financial data
output_dir = 'fin_data'
os.makedirs(output_dir, exist_ok=True)

# Dictionary to hold data for all tickers
all_data = {}

# Loop through each ticker and fetch its data
index = 0
for ticker in tickers:
    print(f"Fetching data for: {ticker}, index={index}")
    index +=1
    try:
        data = fetch_stock_data(ticker)
        all_data[ticker] = data
        # Save data
        data.to_csv(os.path.join(output_dir, f'{ticker}_10_years_data.csv'))
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

Fetching data for: A, index=0
[*********************100%***********************]  1 of 1 completed
Fetching data for: AA, index=1
[*********************100%***********************]  1 of 1 completed
Fetching data for: AACG, index=2
[*********************100%***********************]  1 of 1 completed
Fetching data for: AACI, index=3
[*********************100%***********************]  1 of 1 completed
Fetching data for: AACIW, index=4
[*********************100%***********************]  1 of 1 completed
Fetching data for: AACT, index=5
[*********************100%***********************]  1 of 1 completed
Fetching data for: AADI, index=6
[*********************100%***********************]  1 of 1 completed
Fetching data for: AAGR, index=7
[*********************100%***********************]  1 of 1 completed
Fetching data for: AAGRW, index=8
[*********************100%***********************]  1 of 1 completed
Fetching data for: AAL, index=9
[*********************100%***********************]  1

KeyboardInterrupt: 